In [91]:
import pandas as pd
from pathlib import Path

eda_msas = pd.read_csv("output/analysis-msa/stats_msas.tsv",sep="\t", index_col=False)
eda_msas.drop_duplicates(subset="path_msa", inplace=True,)
eda_msas.tail(20)

,timestamp,path_msa,n_seqs,n_unique_seqs,n_identical_cols,n_cols,perc_identical_cols
37403,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/Cluster_9685.fa,3,1,98,98,100.00
37404,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/Cluster_127.fa,156,8,477,487,97.95
37406,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/GC00008118.fa,4,2,919,957,96.03
37407,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/GC00000791_1.fa,167,31,701,792,88.51
37408,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/GC00000361_2.fa,309,40,838,918,91.29
37409,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/GC00006762_1.fa,1,1,747,747,100.00
37410,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/Cluster_6373.fa,2,2,164,170,96.47
37411,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/GC00007538_1.fa,1,1,1440,1440,100.00
37412,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/GC00004733_1.fa,11,2,392,393,99.75
37413,Mon Dec 12 16:08:40 2022,/home/avila/data/msas/Cluster_11656.fa,1,1,67,67,100.00


In [92]:
valid_msas = eda_msas.query("`n_unique_seqs`>1").copy()
valid_msas.sort_values(by="n_cols", ascending=True, inplace=True, ignore_index=True)
valid_msas["name_msa"] = valid_msas["path_msa"].apply(lambda path_msa: Path(path_msa).stem)

In [93]:
subsample_msas = valid_msas. \
    drop_duplicates(subset=["n_unique_seqs","n_cols"])["name_msa"].\
    sample(n=1000, random_state=1).tolist()
n= len(subsample_msas)
with open("names_msas.txt", "w") as fp:
    for j,name_msa in enumerate(subsample_msas):
        fp.write(subsample_msas[-1]) if j == n-1 else fp.write(name_msa + "\n")

In [38]:
eda_msas.describe()

,n_seqs,n_unique_seqs,n_identical_cols,n_cols,perc_identical_cols
count,37423.000000,37423.000000,37423.000000,37423.000000,37423.000000
mean,52.208562,8.711835,520.590305,550.403495,95.766866
std,97.350875,17.090313,596.563078,635.772178,6.938166
min,1.000000,1.000000,25.000000,30.000000,30.770000
25%,1.000000,1.000000,169.000000,177.000000,93.230000
50%,4.000000,2.000000,312.000000,327.000000,99.800000
75%,40.000000,6.000000,692.000000,738.000000,100.000000
max,2352.000000,366.000000,13377.000000,13377.000000,100.000000


In [41]:
eda_msas.sort_values(by=["n_cols","n_unique_seqs"], ascending=False).head(20)

,timestamp,path_msa,n_seqs,n_unique_seqs,n_identical_cols,n_cols,perc_identical_cols
24715,Mon Dec 12 16:04:54 2022,/home/avila/data/msas/GC00009303.fa,2,1,13377,13377,100.00
2408,Mon Dec 12 16:00:55 2022,/home/avila/data/msas/GC00000160_8.fa,1,1,12159,12159,100.00
15415,Mon Dec 12 16:03:14 2022,/home/avila/data/msas/GC00005508_1.fa,1,1,11046,11046,100.00
13603,Mon Dec 12 16:02:54 2022,/home/avila/data/msas/GC00002971_r1_r1_1.fa,103,48,6556,10695,61.30
20678,Mon Dec 12 16:04:11 2022,/home/avila/data/msas/GC00002971_18.fa,10,2,9753,9813,99.39
6558,Mon Dec 12 16:01:38 2022,/home/avila/data/msas/GC00003866_4.fa,70,23,7956,9669,82.28
20646,Mon Dec 12 16:04:11 2022,/home/avila/data/msas/GC00000082_5.fa,16,7,9609,9618,99.91
29360,Mon Dec 12 16:05:46 2022,/home/avila/data/msas/GC00003866_3.fa,35,15,8651,9507,91.00
8173,Mon Dec 12 16:01:55 2022,/home/avila/data/msas/GC00002971_11.fa,4,2,9446,9507,99.36
21956,Mon Dec 12 16:04:25 2022,/home/avila/data/msas/GC00000082_7.fa,88,40,9197,9498,96.83


In [26]:
eda_msas.describe()

,n_seqs,n_unique_seqs,n_identical_cols,n_cols,perc_identical_cols
count,52266.000000,52266.000000,52266.000000,52266.000000,52266.000000
mean,52.435599,8.735679,521.582999,551.462691,95.757161
std,97.826216,17.202685,600.340150,639.968664,6.951414
min,1.000000,1.000000,25.000000,30.000000,30.770000
25%,1.000000,1.000000,169.000000,177.000000,93.220000
50%,4.000000,2.000000,313.000000,328.000000,99.790000
75%,40.000000,6.000000,692.000000,738.000000,100.000000
max,2352.000000,366.000000,13377.000000,13377.000000,100.000000


## Analysis: old experiment vs new experiment
Algorithm for computing maximal blocks has changed


In [42]:
import pandas as pd
from pathlib import Path

In [43]:
# time max blocks
def load_times(name_msa):
    path_times = [f"output2/block_decomposition/{name_msa}.txt",f"output2/max_blocks/{name_msa}.txt"]
    times = {}

    for path_time in path_times:
        with open(path_time) as fp:
            for line in fp.readlines():
                name, time = line.replace("\n","").split("\t")
                times[name] = float(time)
    
    return times


### old experiments
- `y`: maximal blocks from sub-alignments
- `x`: maximal blocks from the MSA

In [44]:
old_experiments = pd.read_csv("../data/output-old-experiments.tsv", sep="\t")
old_experiments["name_msa"] = old_experiments["path_msa"].apply(lambda path_msa: Path(path_msa).stem)
old_experiments.columns

Index(['Unnamed: 0', 'timestamp_x', 'path_msa', 'path_blocks_x', 'n_seqs_x',
       'n_unique_seqs_x', 'n_cols_x', 'n_max_blocks_x', 't_x',
       'blocks_with_overlap_x', 'inter_between_blocks_x', 'timestamp_y',
       'path_blocks_y', 'n_seqs_y', 'n_unique_seqs_y', 'n_cols_y',
       'n_max_blocks_y', 't_y', 'blocks_with_overlap_y',
       'inter_between_blocks_y', 'diff-max-sub', 'name_msa'],
      dtype='object')

### regarding maximal blocks
- `total_blocks`: number of maximal blocks in the MSA
- `blocks_with_overlap`: blocks with at least one overlap
- `inter_between_blocks`: number of intersections between pairs of blocks (that has to be decomposed)

In [45]:
# maximal blocks
stats_max_blocks = pd.concat(
                            [pd.read_csv(csv, sep="\t", index_col=False) for csv in Path("output2/max_blocks/stats").rglob("*.tsv")],
                            axis=0
                            )
stats_max_blocks["name_msa"] = stats_max_blocks["filename"].apply(lambda filename: Path(filename).stem)
stats_max_blocks


,timestamp,filename,total_blocks,blocks_with_overlap,inter_between_blocks,name_msa
0,Mon Dec 12 15:39:13 2022,output2/max_blocks/Cluster_4804.json,29,239,239,Cluster_4804
0,Mon Dec 12 15:39:10 2022,output2/max_blocks/Cluster_8622.json,25,136,136,Cluster_8622
0,Mon Dec 12 15:39:22 2022,output2/max_blocks/Cluster_7608.json,30,70,70,Cluster_7608
0,Mon Dec 12 15:39:23 2022,output2/max_blocks/GC00003727_1.json,21,25,25,GC00003727_1
0,Mon Dec 12 15:39:25 2022,output2/max_blocks/Cluster_8244.json,3,0,0,Cluster_8244
...,...,...,...,...,...,...
0,Mon Dec 12 15:39:25 2022,output2/max_blocks/GC00008254.json,21,20,20,GC00008254
0,Mon Dec 12 15:39:24 2022,output2/max_blocks/GC00004655.json,2,0,0,GC00004655
0,Mon Dec 12 15:39:11 2022,output2/max_blocks/GC00004466.json,57,350,350,GC00004466
0,Mon Dec 12 15:39:13 2022,output2/max_blocks/GC00004814.json,149,1179,1179,GC00004814


### regarding times
The computation of maximal blocks is as follow: 
1. create a suffix tree as suggested in Alanko's paper
2. find maximal repeats, which will be positional strings for us
3. map the positional strings back to the MSA to identify the maximal blocks
4. decompose the set of maximal blocks to find the set $\mathcal{B}$

then, the reported times are: 
- `t_pos_string`: time to compute positional strings (step 1)
- `t_max_blocks`: time to find maximal blocks (step 2)
- `t_decomp_blocks`: time to decompose pairs of blocks (step 3)

In [46]:
# block decomposition
stats_block_decomp = pd.concat(
                            [pd.read_csv(csv, sep="\t", index_col=False) for csv in Path("output2/block_decomposition/stats").rglob("*.tsv")],
                            axis=0
                            )
stats_block_decomp["name_msa"] = stats_block_decomp["filename"].apply(lambda filename: Path(filename).stem)
stats_block_decomp[["t_decomp_blocks","t_pos_string","t_max_blocks"]] = stats_block_decomp["name_msa"].apply(
    lambda name_msa: pd.Series(load_times(name_msa)))

# to avoid confusion with the total of maximal blocks in stats_max_blocks
stats_block_decomp.rename({"total_blocks": "size_set_B"}, axis=1, inplace=True)

In [47]:
stats_block_decomp.query("name_msa == 'Cluster_13063'")

,timestamp,filename,size_set_B,name_msa,t_decomp_blocks,t_pos_string,t_max_blocks
0,Mon Dec 12 15:39:25 2022,output2/block_decomposition/Cluster_13063.json,7,Cluster_13063,0.000543,0.0294,0.0001


In [48]:
## compare
cols_old = ["name_msa","n_unique_seqs_x","n_seqs_x","n_cols_x","t_x","t_y"] # old experiments
cols_smb = ["name_msa","total_blocks"]#"blocks_with_overlap","inter_between_blocks"] # stats max blocks
cols_sbd = ["name_msa","size_set_B","t_pos_string","t_max_blocks","t_decomp_blocks"]

In [49]:
stats_new_exp = pd.merge(stats_max_blocks[cols_smb],stats_block_decomp[cols_sbd], on="name_msa")

In [50]:
stats_new_exp

,name_msa,total_blocks,size_set_B,t_pos_string,t_max_blocks,t_decomp_blocks
0,Cluster_4804,29,157,0.4290,0.0003,0.017523
1,Cluster_8622,25,98,0.1979,0.0009,0.012874
2,Cluster_7608,30,79,0.1416,0.0001,0.003804
3,GC00003727_1,21,55,0.1752,0.0001,0.002134
4,Cluster_8244,3,3,0.0583,0.0000,0.000524
...,...,...,...,...,...,...
164,GC00008254,21,48,0.0689,0.0001,0.001659
165,GC00004655,2,2,0.1767,0.0000,0.000299
166,GC00004466,57,270,0.3101,0.0007,0.021977
167,GC00004814,149,791,0.9444,0.0014,0.068849


In [51]:
stats_old_exp = old_experiments[cols_old].rename({
    "t_x": "t_max_blocks_msa",
    "t_y": "t_max_blocks_sub",
    "n_unique_seqs_x": "n_unique_seqs",
    "n_seqs_x": "n_seqs_msa",
    "n_cols_x": "n_cols_msa"

}, axis=1)

In [52]:
comparison = pd.merge(stats_old_exp, stats_new_exp, on="name_msa")

In [53]:
# time to compute maximal blocks
# old time / new time
comparison["t_max_blocks_old_over_new"] = comparison.apply(lambda row: row["t_max_blocks_msa"] / (row["t_max_blocks"] + row["t_pos_string"]) ,axis=1)  

In [54]:
comparison.describe()

,n_unique_seqs,n_seqs_msa,n_cols_msa,t_max_blocks_msa,t_max_blocks_sub,total_blocks,size_set_B,t_pos_string,t_max_blocks,t_decomp_blocks,t_max_blocks_old_over_new
count,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000
mean,5.621302,70.710059,128.976331,20.332148,41.860547,26.615385,119.627219,0.185473,0.000707,0.016198,85.194552
std,4.637941,72.176734,58.024108,28.531255,91.060377,36.649401,231.044491,0.223035,0.001669,0.044268,38.599604
min,2.000000,2.000000,31.000000,0.291457,0.030802,1.000000,1.000000,0.007800,0.000000,0.000190,20.244817
25%,2.000000,10.000000,72.000000,2.420653,0.647055,4.000000,5.000000,0.042300,0.000000,0.000389,50.205946
50%,4.000000,34.000000,136.000000,7.811453,5.150988,11.000000,25.000000,0.092900,0.000100,0.001763,84.202646
75%,7.000000,143.000000,183.000000,25.880495,32.429430,33.000000,109.000000,0.251300,0.000700,0.009881,120.410068
max,24.000000,308.000000,216.000000,180.123356,622.211261,197.000000,1526.000000,1.271100,0.016600,0.397701,175.417545


In [55]:
max_ncols = comparison["n_cols_msa"].max()
comparison.query(f"`n_cols_msa`=={max_ncols}")

,name_msa,n_unique_seqs,n_seqs_msa,n_cols_msa,t_max_blocks_msa,t_max_blocks_sub,total_blocks,size_set_B,t_pos_string,t_max_blocks,t_decomp_blocks,t_max_blocks_old_over_new
75,Cluster_4804,7,46,216,56.757239,5.285068,29,157,0.4290,0.0003,0.017523,132.208802
84,GC00001613,24,307,216,180.123356,94.750111,197,1526,1.2711,0.0166,0.397701,139.879907
87,GC00003934_5,2,2,216,11.536539,19.635494,13,13,0.0668,0.0000,0.000537,172.702684


In [56]:
max_nrows = comparison["n_unique_seqs"].max()
comparison.query(f"`n_unique_seqs`=={max_nrows}")

,name_msa,n_unique_seqs,n_seqs_msa,n_cols_msa,t_max_blocks_msa,t_max_blocks_sub,total_blocks,size_set_B,t_pos_string,t_max_blocks,t_decomp_blocks,t_max_blocks_old_over_new
84,GC00001613,24,307,216,180.123356,94.750111,197,1526,1.2711,0.0166,0.397701,139.879907


In [58]:
## sizes
comparison[["n_unique_seqs","n_seqs_msa","n_cols_msa","total_blocks","size_set_B"]].describe()

,n_unique_seqs,n_seqs_msa,n_cols_msa,total_blocks,size_set_B
count,169.000000,169.000000,169.000000,169.000000,169.000000
mean,5.621302,70.710059,128.976331,26.615385,119.627219
std,4.637941,72.176734,58.024108,36.649401,231.044491
min,2.000000,2.000000,31.000000,1.000000,1.000000
25%,2.000000,10.000000,72.000000,4.000000,5.000000
50%,4.000000,34.000000,136.000000,11.000000,25.000000
75%,7.000000,143.000000,183.000000,33.000000,109.000000
max,24.000000,308.000000,216.000000,197.000000,1526.000000
